# App for running GECCO jobs in Galaxy

1. Upload and run workflow.
2. Monitor the job.
3. Receive completion notification with some basic summary provided by Galaxy.

Note: "Receiving" the results (tentatively download) is part of the analysis pipeline.